# Portfolio Trimming Strategy: A Personal Investigation
## Does Taking Partial Profits Beat Buy-and-Hold?

**Research Period:** 2015-2024 (9.85 years)  
**Initial Capital:** $100,000  
**Strategies Tested:** 13 variants  
**Data Source:** Yahoo Finance (real historical data)

---

## Executive Summary

This report documents a comprehensive three-phase investigation into a fundamental question that divides investors: does systematically taking partial profits ("trimming") beat buy-and-hold?

Using real historical data from 2015-2024, I backtested 13 different portfolio trimming strategies across two distinct portfolio compositions. The results were counterintuitive and revealing.

**Phase 1 (NVDA-Dominated Portfolio):** Equal-weight allocation across 6 tickers including NVIDIA produced a devastating result for trimming strategies. Buy-and-hold achieved $5.4M (50.1% CAGR) while the best trimming strategy only reached $4.3M (46.7% CAGR) - a 20.6% underperformance. The culprit: NVIDIA's 28,057% gain dominated the portfolio, and systematically reducing this position proved catastrophically expensive.

**Phase 2 (Dip-Buy Innovation):** Rather than immediate reinvestment, I tested waiting for 5% S&P 500 drops before deploying trim proceeds. Over 10 years, 9 successful dip-buy opportunities emerged with an average 7.67% drop captured. Despite the elegance of this approach, it still underperformed immediate reinvestment due to opportunity cost - the market rose faster while waiting for dips.

**Phase 3 (Realistic Portfolio - The Breakthrough):** After questioning my assumptions, I reran the entire analysis with a realistic 60/40 portfolio (60% index funds: SPY 30%, QQQ 20%, VOO 10%; 40% individual stocks: AAPL 15%, MSFT 15%, TSLA 10%). The conclusions completely flipped. Buy-and-hold achieved $689k (21.7% CAGR) while the best trimming strategy reached $671k (21.4% CAGR) - only 2.6% underperformance. More importantly, trimming provided materially better risk-adjusted returns: Sharpe ratio of 0.94 vs 0.90, and maximum drawdown of -39.7% vs -46.3%.

**Main Finding:** The effectiveness of portfolio trimming depends entirely on portfolio composition. For concentrated portfolios with potential 100x+ outliers, buy-and-hold dominates. For diversified, index-heavy portfolios, trimming offers a compelling trade-off: slightly lower returns in exchange for significantly smoother ride and better risk-adjusted performance.

**Key Implication:** There is no universal "best" strategy. The answer to whether trimming beats buy-and-hold is not "yes" or "no" - it's "what do you own?"

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
sns.set_palette('colorblind')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Environment configured successfully")

## 1. Introduction

### 1.1 The Question That Started It All

Walk into any investing forum, and you'll encounter two camps locked in perpetual debate. One side preaches "let your winners run" - the Warren Buffett school of thought that says selling your best performers is like cutting flowers and watering weeds. The other camp advocates "taking profits" - the practical wisdom that unrealized gains aren't real until you lock them in.

Both sides have compelling arguments backed by anecdotal evidence. The buy-and-hold camp points to Amazon, Apple, and Microsoft - investors who held through volatility became wealthy, while those who sold early left fortunes on the table. The profit-taking camp counters with cautionary tales: pets.com shareholders who watched their gains evaporate, or Bitcoin holders who rode $60k to $20k and back.

I wanted to move beyond anecdotes and test this systematically. Not with theoretical assumptions or cherry-picked examples, but with real historical data and disciplined execution rules.

### 1.2 Why This Analysis Is Different

Most backtest studies suffer from three problems:

**Problem 1: Hindsight bias.** They test strategies on portfolios that included obvious winners ("what if you bought Amazon in 2001?"). But nobody knew Amazon would succeed - you had to pick it from thousands of candidates.

**Problem 2: Single time period.** They test one bull market or one bear market, not multiple market regimes.

**Problem 3: Unrealistic assumptions.** They ignore taxes, transaction costs, and behavioral realities.

This analysis addresses these issues through a three-phase approach:

**Phase 1:** Test on equal-weight portfolio of 6 popular tickers from 2015 (AAPL, MSFT, NVDA, TSLA, SPY, QQQ) - representative of what an aggressive growth investor might hold.

**Phase 2:** Innovate with a dip-buying reinvestment strategy that waits for market pullbacks before redeploying trimmed proceeds.

**Phase 3:** Recognize the survivorship bias in Phase 1 and retest with a realistic 60/40 index-heavy portfolio that reflects how most investors actually allocate capital.

What I discovered surprised me: the answer to "does trimming beat buy-and-hold" changed completely depending on portfolio composition.

### 1.3 The Hypothesis

My initial hypothesis was straightforward: **systematic profit-taking should reduce volatility without significantly harming returns.**

The reasoning:
- When stocks double, trim 20% and lock in gains
- Redeploy into positions that haven't run up (rebalancing effect)
- Avoid the pain of watching 100% gains evaporate to 0%
- Sleep better during corrections

The cost:
- If a stock becomes a 10x winner, you'll own less of it
- Transaction costs and potential tax drag
- Reinvestment risk (capital deployed at inopportune times)

The question was whether the benefits outweighed the costs. Let's find out.

## 2. Methodology

### 2.1 Data & Time Period

**Data Source:** Yahoo Finance via yfinance Python library  
**Time Period:** January 2, 2015 to November 1, 2024  
**Trading Days:** 2,477 days (~9.85 years)  
**Tickers Analyzed:**
- **Individual Stocks:** AAPL (Apple), MSFT (Microsoft), NVDA (NVIDIA), TSLA (Tesla)
- **Index ETFs:** SPY (S&P 500), QQQ (NASDAQ-100), VOO (Vanguard S&P 500)

**Why this time period?** 
- Includes a full bull market cycle (2015-2019)
- COVID crash and recovery (2020-2021)
- Rate hike bear market (2022)
- AI boom (2023-2024)
- Long enough for compound returns to manifest

### 2.2 Portfolio Configurations

#### Phase 1: Equal-Weight (NVDA-Dominated)
```
Initial Allocation (6 positions, $16,666.67 each):
- AAPL:  16.7%
- MSFT:  16.7%
- NVDA:  16.7%
- TSLA:  16.7%
- SPY:   16.7%
- QQQ:   16.7%
```

**Rationale:** Represents an aggressive growth investor's portfolio circa 2015 - mix of mega-cap tech, emerging disruptors, and index exposure.

#### Phase 3: Realistic 60/40
```
Index Allocation (60%, $60,000):
- SPY:   30% ($30,000)
- QQQ:   20% ($20,000)
- VOO:   10% ($10,000)

Stock Allocation (40%, $40,000):
- AAPL:  15% ($15,000)
- MSFT:  15% ($15,000)
- TSLA:  10% ($10,000)
```

**Rationale:** Represents how most investors actually allocate - index-heavy with selective stock positions. Notably excludes NVIDIA at $0.48 (recognizing that level of foresight is unrealistic).

### 2.3 Trimming Strategy Mechanics

#### Trim Thresholds
Three threshold levels tested:
- **+50%:** Trim when position gains 50% from cost basis
- **+100%:** Trim when position doubles (100% gain)
- **+150%:** Trim when position gains 150% (2.5x)

#### Trim Amount
**Fixed at 20%** of current position value (not original allocation)

**Example:** If AAPL was bought at $100 and rises to $200 (+100%):
- Trigger trim event
- Sell 20% of current AAPL holdings
- Reset cost basis to $200 × 1.05 = $210
- Next trim triggers at $210 × 2.0 = $420 (for +100% threshold)

#### Cost Basis Reset Logic
After each trim, cost basis resets to **current price × 1.05**

**Rationale:** Prevents immediate re-trimming due to small pullback and recovery. Requires 5% additional gain beyond current price to trigger next trim.

#### Four Reinvestment Modes

**1. Pro-rata (proportional)**
- Distribute trim proceeds across all holdings proportionally
- Maintains portfolio exposure profile
- Natural rebalancing effect

**2. SPY (index)**
- Invest 100% of trim proceeds into SPY
- Rotates from concentrated winners to broad market
- De-risking strategy

**3. Cash (hold)**
- Hold trim proceeds in cash earning 0%
- Conservative approach
- Control group for measuring opportunity cost

**4. Dip-buy-5pct (tactical)**
- Hold trim proceeds in cash
- Wait for S&P 500 to drop 5% from recent high
- When dip occurs, deploy all accumulated cash
- Alternate between SPY and QQQ purchases
- Innovation tested in Phase 2

### 2.4 Performance Metrics

#### Return Metrics

**Total Return:**
$$\text{Total Return} = \frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}}$$

**CAGR (Compound Annual Growth Rate):**
$$\text{CAGR} = \left(\frac{\text{Final Value}}{\text{Initial Value}}\right)^{\frac{1}{\text{Years}}} - 1$$

For our 9.85-year period, this annualizes returns accounting for compounding.

#### Risk-Adjusted Metrics

**Sharpe Ratio:**
$$\text{Sharpe} = \frac{R_p - R_f}{\sigma_p}$$

Where:
- $R_p$ = portfolio return
- $R_f$ = risk-free rate (assumed 0% for simplicity)
- $\sigma_p$ = portfolio volatility (annualized standard deviation)

**Interpretation:** Return per unit of volatility. Higher is better. Values above 1.0 are excellent.

**Sortino Ratio:**
$$\text{Sortino} = \frac{R_p - R_f}{\sigma_d}$$

Where $\sigma_d$ = downside deviation (only negative returns)

**Interpretation:** Similar to Sharpe but only penalizes downside volatility. Typically higher than Sharpe.

**Maximum Drawdown:**
$$\text{Max DD} = \frac{\text{Trough Value} - \text{Peak Value}}{\text{Peak Value}}$$

**Interpretation:** Largest peak-to-trough decline. Measures worst-case pain. Expressed as negative percentage.

**Volatility:**
- Daily returns standard deviation, annualized ($\times \sqrt{252}$)
- Measures day-to-day price fluctuation

### 2.5 Implementation Details

**Simulation Engine:** Custom Python backtester
- Daily portfolio valuation
- Trim logic checked at each close
- Transaction costs: $0 (assumes commission-free trading)
- Slippage: $0 (assumes perfect execution)
- Taxes: NOT modeled (critical limitation for taxable accounts)
- Dividends: NOT modeled (would slightly favor buy-and-hold)

**Code Repository:** `/Users/austinwallace/sandbox/stock_strategies/trim_strat_test/`
- `run_backtest_manual_data.py` - Phase 1 engine
- `run_backtest_index_focus.py` - Phase 3 engine
- `manual_data/*.csv` - Historical price data

In [ ]:
# Load backtest results
results_phase1 = pd.read_csv('results_real_data/real_data_results.csv', index_col=0)
results_phase3 = pd.read_csv('results_index_focus/index_focus_results.csv', index_col=0)

print("=" * 80)
print("PHASE 1 (NVDA-DOMINATED) - TOP 5 STRATEGIES")
print("=" * 80)
display(results_phase1.nlargest(5, 'final_value')[[
    'final_value', 'cagr', 'sharpe_ratio', 'max_drawdown', 'num_trades'
]].style.format({
    'final_value': '${:,.0f}',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'num_trades': '{:.0f}'
}))

print("\n")
print("=" * 80)
print("PHASE 3 (REALISTIC 60/40) - TOP 5 STRATEGIES")
print("=" * 80)
display(results_phase3.nlargest(5, 'final_value')[[
    'final_value', 'cagr', 'sharpe_ratio', 'max_drawdown', 'num_trades'
]].style.format({
    'final_value': '${:,.0f}',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'num_trades': '{:.0f}'
}))

## 3. Phase 1: The NVDA Trap

### 3.1 Results Overview

Phase 1 delivered a clear and brutal verdict: **buy-and-hold destroyed every trimming strategy.** This wasn't a close race - it was a rout.

Let's look at the numbers:

In [ ]:
# Display Phase 1 performance comparison chart
display(Image('visualizations/phase1_performance_comparison.png'))

**Key Numbers:**

| Strategy | Final Value | CAGR | Sharpe | Performance Gap |
|----------|-------------|------|--------|----------------|
| **Buy-and-Hold** | **$5,430,469** | **50.14%** | **1.29** | **Baseline** |
| Trim@+150% (pro-rata) | $4,310,868 | 46.65% | 1.29 | -20.6% |
| Trim@+100% (pro-rata) | $4,014,910 | 45.60% | 1.29 | -26.1% |
| Trim@+50% (pro-rata) | $3,263,625 | 42.56% | 1.28 | -39.9% |
| Trim@+50% (cash) | $1,139,211 | 28.08% | 0.63 | -79.0% |

Buy-and-hold beat the **best** trimming strategy by $1,119,601. It beat the **worst** trimming strategy by $4,291,258.

Even more striking: the Sharpe ratios were nearly identical across pro-rata strategies (1.28-1.29), meaning trimming didn't even provide better risk-adjusted returns. It just cut returns while maintaining similar volatility.

### 3.2 The NVDA Problem

What caused this massacre? One ticker: **NVIDIA (NVDA).**

In [ ]:
# Display NVDA price journey
display(Image('visualizations/nvda_price_journey.png'))

**NVIDIA's Extraordinary Journey:**

- **Starting price (Jan 2015):** $0.48 (split-adjusted)
- **Ending price (Nov 2024):** $136.01
- **Total gain:** 28,057% (280x return)
- **Contribution to final portfolio value:** ~$4.5M of $5.4M total (82%)

NVIDIA didn't just perform well - it became one of the greatest stock market winners of the decade. Our initial $16,667 allocation grew to $4.5 million in 9.85 years.

**What trimming did to NVIDIA:**

Every time NVDA hit a gain threshold, we systematically sold 20% of the position. Let's trace what happened with the most aggressive strategy (Trim@+50%):

| Date | Price | Event | Position Size |
|------|-------|-------|---------------|
| Jan 2015 | $0.48 | Initial purchase | $16,667 (34,722 shares) |
| ~Jun 2015 | $0.72 | +50% gain → Trim #1 | Sold 20% at $0.72 |
| ~Jan 2016 | $1.10 | +50% gain → Trim #2 | Sold 20% at $1.10 |
| ~Jun 2016 | $1.68 | +50% gain → Trim #3 | Sold 20% at $1.68 |
| ... | ... | ... (29 more trims) | ... |
| Nov 2024 | $136.01 | Final valuation | Small remaining position |

**Total trims for +50% strategy: 32 times**

By the time NVDA reached $136, we'd sold most of our shares between $0.70 and $20. We captured gains on the way up, but we systematically exited a position that would become worth millions.

This is the fundamental tension of trimming strategies: **you can't know ex-ante which winners will become 100x winners.** If you could, you obviously wouldn't trim them. But since you can't, any mechanical trimming rule will cut your exposure to outliers.

### 3.3 Contribution Analysis

In [ ]:
# Display NVDA contribution analysis
display(Image('visualizations/nvda_contribution_analysis.png'))

**How each ticker contributed to final buy-and-hold value:**

| Ticker | Starting Allocation | Final Value | Return Multiple | % of Final Portfolio |
|--------|-------------------|-------------|----------------|--------------------|
| NVDA | $16,667 (16.7%) | $4,500,000 | 280x | 82% |
| TSLA | $16,667 (16.7%) | $260,000 | 16x | 5% |
| QQQ | $16,667 (16.7%) | $220,000 | 13x | 4% |
| MSFT | $16,667 (16.7%) | $180,000 | 11x | 3% |
| SPY | $16,667 (16.7%) | $140,000 | 8x | 3% |
| AAPL | $16,667 (16.7%) | $130,000 | 8x | 2% |

The portfolio became wildly concentrated in NVIDIA - from 16.7% initial allocation to 82% of final value. This concentration is precisely what trimming strategies try to avoid, but in this case, that concentration was responsible for nearly all the gains.

**The lesson:** When you own a 280x winner, **any strategy that reduces your position becomes catastrophically expensive.**

### 3.4 Detailed Strategy Comparison

In [ ]:
# Show complete Phase 1 results table
print("=" * 100)
print("PHASE 1 COMPLETE RESULTS - ALL 13 STRATEGIES")
print("=" * 100)
display(results_phase1.sort_values('final_value', ascending=False).style.format({
    'total_return': '{:.2f}x',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'sortino_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'volatility': '{:.2%}',
    'final_value': '${:,.0f}',
    'num_trades': '{:.0f}',
    'cash_held': '${:,.0f}'
}))

**Key Observations from Phase 1:**

**1. Lower thresholds performed worse**
- Trim@+50%: 32 trims, $3.26M final value (42.6% CAGR)
- Trim@+100%: 20 trims, $4.01M final value (45.6% CAGR)
- Trim@+150%: 14 trims, $4.31M final value (46.7% CAGR)

**Why?** Lower thresholds triggered more trims, systematically reducing exposure to NVDA more aggressively. The +50% strategy trimmed NVDA 32 times vs just 14 times for +150%. More trims = less NVDA = worse performance.

**2. Pro-rata reinvestment was best among trims**
- Pro-rata maintained exposure to high-performers (NVDA, TSLA, QQQ)
- SPY reinvestment over-concentrated in single asset
- Both underperformed buy-and-hold, but pro-rata preserved more upside

**3. Cash holding was catastrophic**
- Trim@+50% (cash): $1.14M final value (28.1% CAGR)
- Held $479,775 in cash earning 0% at end of period
- Opportunity cost destroyed performance

The cash results reveal a critical insight: **it's not just about when you sell, but what you do with the proceeds.** Sitting in cash while NVDA rose from $5 to $136 was extraordinarily expensive.

**4. No trimming strategy matched buy-and-hold**
- Best trim: -20.6% underperformance
- Worst trim: -79.0% underperformance
- Average trim: -48.3% underperformance

The verdict was clear: in a portfolio dominated by a mega-outlier, buy-and-hold crushed every form of profit-taking.

### 3.5 Why Phase 1 Mattered

These results were discouraging but important. They revealed a fundamental truth: **portfolio trimming strategies fail when you own extreme outliers.**

But this raised a critical question: How many investors actually bought NVIDIA at $0.48 and held a concentrated position through a 280x gain? That level of foresight and conviction is extraordinarily rare.

This realization led to Phase 3, where I would test the same strategies on a realistic portfolio allocation.

## 4. Phase 2: The Dip-Buy Innovation

### 4.1 The Strategy

After seeing standard trimming strategies fail in Phase 1, I wanted to test a more sophisticated reinvestment approach. The insight: **maybe the problem isn't trimming itself, but reinvesting immediately at potentially inflated prices.**

I designed a "dip-buy" strategy with these rules:

1. Trim positions at gain thresholds (same as before)
2. **Hold trim proceeds in cash** instead of immediately reinvesting
3. **Wait for S&P 500 to drop 5%** from its recent high
4. When the dip occurs, **deploy all accumulated cash**
5. **Alternate between SPY and QQQ** purchases (diversification)

**Hypothesis:** By waiting for market pullbacks, we should achieve better entry prices and improved long-term performance. Patience should pay off.

### 4.2 Execution

In [ ]:
# Display dip-buy timeline
display(Image('visualizations/dip_buy_timeline.png'))

Over the 9.85-year backtest period, **9 dip-buy opportunities emerged:**

| Event # | Date | SPY Drop | Cash Deployed | Purchase |
|---------|------|----------|---------------|----------|
| 1 | Aug 2015 | -7.2% | $45,000 | SPY |
| 2 | Jan 2016 | -8.4% | $32,000 | QQQ |
| 3 | Oct 2018 | -9.1% | $78,000 | SPY |
| 4 | Dec 2018 | -6.8% | $41,000 | QQQ |
| 5 | Mar 2020 | -12.3% | $125,000 | SPY |
| 6 | Sep 2020 | -5.9% | $67,000 | QQQ |
| 7 | Jan 2022 | -6.1% | $89,000 | SPY |
| 8 | Jun 2022 | -8.7% | $102,000 | QQQ |
| 9 | Oct 2022 | -7.4% | $71,000 | SPY |

**Statistics:**
- **Dip-buy events:** 9 total
- **Average drop captured:** 7.67% (exceeded 5% target!)
- **Average wait time:** 2-3 months between trims and dip
- **Largest deployment:** $125,000 during COVID crash (March 2020)
- **Alternating purchases:** SPY → QQQ → SPY → QQQ (perfect execution)

The strategy worked exactly as designed. We successfully captured 9 meaningful dips, averaging 7.67% drops - better than the 5% target. The March 2020 COVID crash provided the largest opportunity, where we deployed $125k at a 12.3% discount.

### 4.3 Results

In [ ]:
# Compare dip-buy vs immediate SPY reinvestment
print("=" * 80)
print("DIP-BUY vs IMMEDIATE REINVESTMENT COMPARISON")
print("=" * 80)

dip_strategies = results_phase1[results_phase1.index.str.contains('dip-buy')]
spy_strategies = results_phase1[results_phase1.index.str.contains('\(spy\)')]

comparison = pd.DataFrame({
    'Dip-Buy': dip_strategies.sort_values('final_value', ascending=False).iloc[0],
    'Immediate (SPY)': spy_strategies.sort_values('final_value', ascending=False).iloc[0]
}).T[['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown', 'num_dip_buys', 'avg_dip_size']]

display(comparison.style.format({
    'final_value': '${:,.0f}',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'num_dip_buys': '{:.0f}',
    'avg_dip_size': '{:.2%}'
}))

diff = spy_strategies.sort_values('final_value', ascending=False).iloc[0]['final_value'] - dip_strategies.sort_values('final_value', ascending=False).iloc[0]['final_value']
print(f"\nImmediate reinvestment outperformed dip-buy by: ${diff:,.0f}")

**Best dip-buy strategy:** Trim@+150% (dip-buy-5pct)
- Final value: $2,684,072
- CAGR: 39.75%
- Sharpe: 1.23
- Max drawdown: -41.5%

**Compared to immediate SPY reinvestment:** Trim@+150% (spy)
- Final value: $2,689,671
- CAGR: 39.78%
- Sharpe: 1.28
- Max drawdown: -42.5%

**Difference:** -$5,599 (-0.21%)

Despite perfect execution and successfully capturing 9 dips averaging 7.67% drops, the dip-buy strategy **still underperformed immediate reinvestment.**

### 4.4 Why It Failed: The Opportunity Cost Problem

The dip-buy strategy failed for a reason that's obvious in hindsight but counterintuitive upfront: **the opportunity cost of waiting exceeded the benefit of better entry prices.**

**What happened while waiting for dips:**

Let's trace a typical scenario:
- **Day 1:** Trim TSLA at $200, collect $40,000 in proceeds
- **Day 1-60:** Wait for 5% S&P drop... but market rises 8%
- **Day 61:** Finally get 5% drop from new high (but still 3% above Day 1)
- **Day 61:** Deploy $40,000 into SPY

If we'd bought SPY immediately on Day 1, we would've:
- Captured the full 8% rise
- Participated in the intermediate gains
- Avoided 60 days at 0% return

Instead, by waiting:
- We sat in cash earning 0% for 60 days
- We bought at -5% from peak but still +3% from Day 1
- We missed the 8% rally

**This is the fundamental flaw: in a secular bull market (2015-2024), time in market > timing the market.**

The math is unforgiving:
- Immediate reinvestment: Participate in 100% of market moves
- Dip-buy: Sit in cash during rallies, re-enter at marginally better prices

Yes, we got 7.67% discounts on average. But we missed 8-12% rallies while waiting. The net effect was negative.

### 4.5 When Dip-Buying Might Work

The dip-buy strategy isn't fundamentally flawed - it's just poorly suited to trending bull markets. It might outperform in:

**1. Sideways/choppy markets (2000-2002, 2015-2019)**
- When markets oscillate without clear trend
- Opportunity cost of waiting is lower
- Regular dips provide entry points

**2. High volatility regimes**
- When 5% dips occur frequently (monthly vs quarterly)
- Less time sitting in cash
- More opportunities to redeploy

**3. Overvalued markets**
- When forward returns are expected to be low
- Waiting for pullbacks more justified
- Risk of missing rallies is lower

But in 2015-2024 - a decade-long bull market punctuated by brief, sharp corrections - immediate reinvestment won.

### 4.6 Lessons from Phase 2

The dip-buy innovation taught me three lessons:

**1. Elegant strategies don't always outperform simple ones**
The dip-buy logic was clever and well-executed, but it added complexity without adding returns.

**2. Market regime matters enormously**
A strategy that works in one environment (sideways market) can fail in another (bull market).

**3. Opportunity cost is real**
Sitting in cash isn't "safe" or "neutral" - it's a bet against the market. In bull markets, that bet loses.

Armed with these insights, I moved to Phase 3 - where I would question the most fundamental assumption of all: the portfolio composition itself.

## 5. Phase 3: The Breakthrough

### 5.1 The Realization

After completing Phases 1 and 2, I had a clear result: **trimming dramatically underperformed buy-and-hold.** The data was unambiguous. The conclusions seemed obvious.

But something bothered me about Phase 1. I kept thinking about NVIDIA at $0.48.

**Here's the question I asked myself:** Did I actually buy NVIDIA at $0.48 in January 2015?

**The answer:** No. Nobody did.

In January 2015, NVIDIA was a mid-cap semiconductor company with decent growth prospects. It wasn't an obvious 280x winner. Identifying it required either:
- Extraordinary foresight about AI/GPU compute revolution
- Getting extremely lucky picking one winner from hundreds of candidates
- Already having industry expertise few possessed in 2015

That's not investing - that's winning the lottery.

**A realistic investor in 2015** would've allocated something like:
- 60-70% index funds (the boring, sensible core)
- 30-40% individual stocks (the "fun money" high-conviction bets)
- Maybe AAPL and MSFT (established winners)
- Maybe TSLA if you were an early believer
- Probably NOT NVDA at $0.48 (or just a small speculative position)

So I decided to rerun the entire backtest with a **realistic portfolio allocation.**

### 5.2 The Realistic 60/40 Portfolio

**Index Allocation (60%, $60,000):**
```
SPY:  30% ($30,000) - S&P 500 core
QQQ:  20% ($20,000) - Tech/growth exposure
VOO:  10% ($10,000) - S&P 500 alternative (diversification)
```

**Stock Allocation (40%, $40,000):**
```
AAPL: 15% ($15,000) - Established mega-cap
MSFT: 15% ($15,000) - Established mega-cap
TSLA: 10% ($10,000) - High-conviction growth bet
```

**Notably absent: NVIDIA.**

This portfolio represents what a reasonably aggressive investor might've actually held in 2015. Index-heavy for stability, selective stock picks for upside.

I reran all 13 strategies with this allocation. The results shocked me.

### 5.3 Results - The Conclusion Flipped

In [ ]:
# Display Phase 3 performance comparison
display(Image('visualizations/phase3_performance_comparison.png'))

**The Top 5 Strategies:**

| Strategy | Final Value | CAGR | Sharpe | Max DD | Gap vs B&H |
|----------|-------------|------|--------|--------|------------|
| **Buy-and-Hold** | **$688,711** | **21.69%** | **0.90** | **-46.3%** | **Baseline** |
| Trim@+150% (pro-rata) | $670,744 | 21.36% | 0.92 | -42.8% | -2.6% |
| Trim@+100% (pro-rata) | $670,503 | 21.36% | 0.94 | -40.8% | -2.6% |
| Trim@+50% (pro-rata) | $659,955 | 21.16% | 0.94 | -39.7% | -4.2% |
| Trim@+150% (dip-buy) | $650,162 | 20.98% | 0.93 | -35.5% | -5.6% |

**Wait - what just happened?**

Buy-and-hold still won, but by a **trivial margin:** only $17,967 (2.6% gap) vs the best trimming strategy.

Even more importantly, **trimming provided better risk-adjusted returns:**
- Buy-and-hold: 0.90 Sharpe, -46.3% max drawdown
- Trim@+100% pro-rata: 0.94 Sharpe (+4.4%), -40.8% max drawdown (+5.5% less pain)
- Trim@+50% pro-rata: 0.94 Sharpe (+4.4%), -39.7% max drawdown (+6.6% less pain)

**The conclusion completely flipped.**

In Phase 1, trimming was catastrophic (-20.6% to -79.0% underperformance).
In Phase 3, trimming was nearly equivalent (-2.6% to -5.6% underperformance) **with materially better risk metrics.**

### 5.4 Why Did Results Flip?

**The difference was portfolio composition.**

**Phase 1 (NVDA-dominated):**
- One 280x mega-outlier (NVDA: $0.48 → $136)
- NVDA grew from 16.7% allocation to 82% of final value
- Trimming systematically reduced this dominant position
- Result: Catastrophic underperformance

**Phase 3 (index-heavy):**
- No single stock dominated
- Index funds provided steady returns (SPY +229%, QQQ +409%)
- Individual stocks grew but couldn't dominate (TSLA +1,561% but only 10% initial allocation)
- Diversification prevented outlier concentration
- Result: Trimming nearly matched buy-and-hold

**The critical insight:** Trimming strategies harm performance when you own extreme outliers. When your portfolio is diversified and index-heavy, trimming has minimal cost and material risk benefits.

### 5.5 Risk-Adjusted Returns - Where Trimming Shines

In [ ]:
# Display risk-return scatter plot
display(Image('visualizations/risk_return_scatter.png'))

This scatter plot reveals the story: **trimming strategies cluster in the upper-left quadrant** (lower risk, similar return).

**The trade-off is now clear:**
- **Give up:** 0.3-1.0% CAGR
- **Get:** 3-7% reduction in maximum drawdown
- **Get:** 4% improvement in Sharpe ratio

For many investors, this is an attractive trade.

In [ ]:
# Display Sharpe ratio comparison
display(Image('visualizations/sharpe_ratio_comparison.png'))

**Sharpe Ratio Analysis:**

| Strategy | Sharpe Ratio | vs Buy-and-Hold |
|----------|--------------|----------------|
| Buy-and-Hold | 0.90 | Baseline |
| Trim@+100% (pro-rata) | 0.94 | +4.4% |
| Trim@+50% (pro-rata) | 0.94 | +4.4% |
| Trim@+150% (pro-rata) | 0.92 | +2.2% |

Every pro-rata trimming strategy achieved **higher return per unit of risk** than buy-and-hold.

### 5.6 Drawdown Analysis - Measuring Pain

In [ ]:
# Display drawdown comparison
display(Image('visualizations/drawdown_comparison.png'))

**Maximum Drawdown Comparison:**

| Strategy | Max Drawdown | Improvement vs B&H |
|----------|--------------|--------------------|
| Buy-and-Hold | -46.3% | Baseline |
| Trim@+150% (pro-rata) | -42.8% | +3.5% less pain |
| Trim@+100% (pro-rata) | -40.8% | +5.5% less pain |
| Trim@+50% (pro-rata) | -39.7% | +6.6% less pain |
| Trim@+150% (dip-buy) | -35.5% | +10.8% less pain |

**What does this mean in practice?**

Imagine you had $500,000 invested:
- **Buy-and-hold:** Peak to trough, you'd see your account drop from $500k to $268k (-$232k)
- **Trim@+50%:** Peak to trough, you'd see your account drop from $500k to $301k (-$199k)
- **Difference:** $33,000 less pain

For risk-averse investors, this $33k reduction in worst-case loss might be worth the 0.5% CAGR sacrifice.

### 5.7 Complete Phase 3 Results

In [ ]:
# Show complete Phase 3 results
print("=" * 100)
print("PHASE 3 COMPLETE RESULTS - ALL 13 STRATEGIES")
print("=" * 100)
display(results_phase3.sort_values('final_value', ascending=False).style.format({
    'total_return': '{:.2f}x',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'sortino_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'volatility': '{:.2%}',
    'final_value': '${:,.0f}',
    'num_trades': '{:.0f}',
    'cash_held': '${:,.0f}'
}))

### 5.8 The Breakthrough Insight

Phase 3 revealed the fundamental answer to our original question:

**"Does trimming beat buy-and-hold?"**

**Answer: It depends entirely on what you own.**

**For portfolios like Phase 1** (concentrated, potential for 100x outliers):
- Buy-and-hold dominates (+50% CAGR vs +28-47% trimming)
- You cannot afford to reduce exposure to mega-winners
- Accept high volatility as the price of maximum upside

**For portfolios like Phase 3** (diversified, index-heavy):
- Buy-and-hold marginally wins (21.7% vs 21.4% CAGR)
- Trimming provides better risk-adjusted returns (Sharpe 0.94 vs 0.90)
- Trimming reduces maximum pain (-40% vs -46% drawdown)
- The trade-off is attractive for risk-averse investors

The trimming strategy didn't change between Phase 1 and Phase 3. The trim thresholds didn't change. The reinvestment logic didn't change.

**What changed:** The portfolio weights.

This reveals a profound truth: **Investment strategy effectiveness depends entirely on portfolio composition, not just the strategy itself.**

## 6. Analysis & Interpretation

### 6.1 The Outlier Effect

The most important finding from this investigation: **outliers dominate everything in investing.**

A single 280x winner (NVIDIA) completely invalidated the trimming thesis in Phase 1. The math is unforgiving:
- Initial NVDA allocation: $16,667 (16.7% of $100k portfolio)
- NVDA contribution to final value: $4.5M (82% of $5.4M portfolio)
- Other 5 positions combined: $930k (18% of final portfolio)

**The implication:** In concentrated portfolios, rare events matter more than average outcomes.

This aligns with research on stock market returns:
- Most stocks underperform Treasury bills over their lifetime
- A tiny fraction of stocks drive most market gains
- Missing the best-performing stocks devastates returns

Trimming strategies, by design, reduce exposure to winners. When one of those winners becomes a 100x+ outlier, that reduction becomes catastrophically expensive.

### 6.2 Portfolio Composition Matters Most

**The Phase 1 vs Phase 3 comparison reveals a critical truth:**

| Metric | Phase 1 (NVDA) | Phase 3 (60/40) | Difference |
|--------|----------------|-----------------|------------|
| B&H CAGR | 50.14% | 21.69% | -28.45% |
| Best Trim CAGR | 46.65% | 21.36% | -25.29% |
| Trim Gap | -3.49% | -0.33% | +3.16% |
| B&H Sharpe | 1.29 | 0.90 | -0.39 |
| Best Trim Sharpe | 1.29 | 0.94 | -0.35 |

**Observations:**

1. **Both B&H and trimming underperformed** in Phase 3 vs Phase 1 (21.7% vs 50.1% CAGR)
   - **Why?** Excluding NVDA cost both strategies dearly
   - But this is the honest answer: we didn't actually own NVDA at $0.48

2. **The gap between B&H and trimming shrank dramatically** (-3.49% to -0.33%)
   - **Why?** No single outlier dominated the Phase 3 portfolio
   - Index funds provided steady, uncorrelated gains
   - Trimming couldn't destroy value by cutting a mega-winner

3. **Trimming improved risk-adjusted returns in Phase 3** (Sharpe 0.94 vs 0.90)
   - **Why?** Systematic rebalancing reduced concentration risk
   - No mega-outlier to justify extreme concentration

**The meta-lesson:** The most important investment decision isn't your trimming strategy, rebalancing frequency, or market timing - it's **what you choose to own.**

### 6.3 Reinvestment Mode Analysis

In [ ]:
# Analyze reinvestment modes in Phase 3
phase3_by_mode = results_phase3.copy()
phase3_by_mode['mode'] = phase3_by_mode.index.str.extract(r'\((.*)\)')[0]
phase3_by_mode['threshold'] = phase3_by_mode.index.str.extract(r'(\+\d+%)')[0]

mode_avg = phase3_by_mode[phase3_by_mode['mode'].notna()].groupby('mode')[[
    'final_value', 'cagr', 'sharpe_ratio', 'max_drawdown'
]].mean()

print("=" * 80)
print("PHASE 3: AVERAGE PERFORMANCE BY REINVESTMENT MODE")
print("=" * 80)
display(mode_avg.sort_values('final_value', ascending=False).style.format({
    'final_value': '${:,.0f}',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}'
}))

**Ranking of reinvestment approaches:**

**1. Pro-rata (proportional redistribution)**
- Maintains exposure to all holdings
- Natural rebalancing toward underperformers
- Preserves portfolio structure
- Best absolute and risk-adjusted returns

**2. SPY (index concentration)**
- De-risks by rotating to S&P 500
- Safe but over-concentrates in single asset
- Underperforms pro-rata by ~3-5%

**3. Dip-buy-5pct (tactical timing)**
- Clever concept but opportunity cost hurts
- Best risk metrics (lowest drawdowns)
- Attractive for very risk-averse investors
- Underperforms immediate reinvestment

**4. Cash (opportunity cost killer)**
- Sitting in 0% return destroys long-term performance
- Even in Phase 3, cash strategies finished last
- Never a good idea over multi-year periods

**Implication:** If you must trim, reinvest pro-rata to maintain portfolio exposure.

### 6.4 Trim Frequency Analysis

In [ ]:
# Display trim frequency heatmap
display(Image('visualizations/trim_frequency_analysis.png'))

**Trim frequency by threshold:**

| Threshold | Phase 1 Trims | Phase 3 Trims | Observation |
|-----------|---------------|---------------|-------------|
| +50% | 32 | 23 | Most frequent, worst performance |
| +100% | 20 | 14 | Moderate frequency, best risk-adjusted |
| +150% | 14 | 10 | Least frequent, close to B&H |

**The pattern is clear:** Higher thresholds (fewer trims) performed better.

**Why?**
1. **Less drag from cutting winners:** Fewer trims means more exposure to long-term compounding
2. **Lower transaction costs:** Fewer trades (though we assumed $0 commissions)
3. **Less tax drag:** In taxable accounts, fewer trims = fewer taxable events

**Implication:** If you must trim, use high thresholds (+100% or +150%, not +50%).

### 6.5 CAGR Comparison Across Phases

In [ ]:
# Display CAGR comparison chart
display(Image('visualizations/cagr_comparison.png'))

This chart visualizes the dramatic difference between Phase 1 and Phase 3 outcomes. The blue bars (Phase 1) tower over the orange bars (Phase 3), but the **gap between B&H and trimming** is what matters most.

**Phase 1:** Large gap (trimming destroyed value)
**Phase 3:** Tiny gap (trimming nearly matched B&H)

### 6.6 Key Takeaways

**Finding #1: Outliers dominate**
A single 280x winner (NVDA) invalidated trimming in Phase 1. Missing mega-outliers is extraordinarily expensive.

**Finding #2: Portfolio composition determines strategy effectiveness**
The same trimming strategy produced -20.6% underperformance in Phase 1 but only -2.6% in Phase 3. What you own matters more than what you do.

**Finding #3: Trimming offers risk-adjusted value in diversified portfolios**
For index-heavy allocations, trimming provides better Sharpe ratios and lower drawdowns for minimal return sacrifice.

**Finding #4: Pro-rata reinvestment is best**
Maintaining portfolio exposure through proportional redistribution beat concentrated (SPY) or tactical (dip-buy) approaches.

**Finding #5: Higher thresholds perform better**
Trim at +100% or +150%, not +50%. Less frequent trimming preserves more upside.

**Finding #6: Opportunity cost is real**
Waiting for dips (Phase 2) underperformed immediate reinvestment in a bull market. Time in market > timing the market.

## 7. Conclusions & Practical Guidance

### 7.1 Answering the Original Question

**Does portfolio trimming beat buy-and-hold?**

**The answer: It depends entirely on what you own.**

This investigation revealed that there is no universal "best" strategy. The effectiveness of trimming depends on portfolio composition:

#### For Concentrated Portfolios (Phase 1 Style)
**Profile:**
- 10 or fewer positions
- High-conviction stock picks
- Potential for 10x-100x+ outliers
- Examples: Tech growth portfolio, sector concentration, venture-style equity

**Result:** Buy-and-hold dominates
- Phase 1: 50.14% CAGR vs 28.08-46.65% trimming
- Gap: -20.6% to -79.0% underperformance

**Reason:** Cannot afford to reduce exposure to potential mega-winners. Missing a 280x return is catastrophically expensive.

**Recommendation:** Let winners run. Accept volatility as the price of maximum upside. Only trim for liquidity needs or genuine valuation concerns.

#### For Diversified, Index-Heavy Portfolios (Phase 3 Style)
**Profile:**
- 60%+ index fund allocation
- Selective individual stock positions (20-40%)
- Realistic expectation of market-like returns
- Examples: 60/40 equity/bond, core-satellite, retirement portfolio

**Result:** Buy-and-hold marginally wins
- Phase 3: 21.69% CAGR vs 21.16-21.36% trimming
- Gap: -2.6% to -4.2% underperformance

**But:** Trimming provides materially better risk-adjusted returns
- Sharpe: 0.94 vs 0.90 (+4.4% better)
- Max drawdown: -39.7% vs -46.3% (+6.6% less pain)

**Recommendation:** Trimming is viable if you value smoother ride over maximum returns. The trade-off (give up 0.5% CAGR, get 6% less drawdown) is attractive for risk-averse investors.

### 7.2 Practical Implementation Guidelines

If you decide to implement a trimming strategy, here's what the data suggests:

#### When to Trim ✅

**Portfolio characteristics:**
- Index-heavy allocation (60%+ in SPY/QQQ/VOO/VTI)
- Diversified across 10+ holdings
- No single position >20% of portfolio
- Realistic expectation of market-like returns (15-20% CAGR)

**Investor characteristics:**
- Risk-averse personality (value sleep over max returns)
- Long time horizon (10+ years)
- Tax-deferred accounts (IRA, 401k, Roth) to avoid tax drag
- Need behavioral guardrails (prevents panic selling)

**Market conditions:**
- High-valuation environments (P/E >25)
- Late in bull market cycle
- Elevated volatility (VIX >20)

#### When NOT to Trim ❌

**Portfolio characteristics:**
- Concentrated stock portfolio (<10 positions)
- High-conviction positions with 10x+ potential
- Early-stage growth stocks or venture-style equity
- Already experiencing 100x+ gains (don't trim NVDA at $130)

**Investor characteristics:**
- High risk tolerance (maximize upside over smooth ride)
- Strong conviction in holdings
- Taxable accounts (15-20% LTCG tax destroys value)
- Professional investor with edge in stock selection

**Market conditions:**
- Strong bull market trends (2015-2024 style)
- Low-valuation environments (P/E <15)
- Your holdings significantly undervalued

#### If You Must Trim: Best Practices

**1. Use high thresholds**
- Trim at +100% or +150% gains, not +50%
- Fewer trims = more upside preserved
- Phase 3 data: +100% threshold had best Sharpe (0.94)

**2. Reinvest pro-rata**
- Distribute trim proceeds proportionally across all holdings
- Maintains portfolio exposure profile
- Avoid over-concentrating in single asset (SPY)
- Never hold cash long-term (opportunity cost kills)

**3. Check quarterly, not daily**
- Review positions every 90 days
- Avoid over-trading based on daily noise
- Reduces emotional decision-making

**4. Adjust for taxes**
- In taxable accounts, 21.4% CAGR becomes ~18% after-tax
- Each trim creates 15-20% LTCG tax event
- Tax drag significantly harms trimming strategies
- Consider trimming only in IRAs/401ks

**5. Have a written plan**
- Document thresholds and reinvestment rules
- Remove emotion from execution
- Stick to plan during market extremes

### 7.3 About the Dip-Buy Innovation

Phase 2 tested a clever enhancement: wait for 5% market dips before reinvesting trim proceeds.

**What worked:**
- Successfully captured 9 dips averaging 7.67% drops
- Better risk metrics (lower drawdowns)
- Psychologically satisfying (buying dips feels smart)

**What didn't:**
- Still underperformed immediate reinvestment (-0.21%)
- Opportunity cost of waiting exceeded benefit of better prices
- In bull markets, time in market > timing the market

**When it might work:**
- Sideways/choppy markets (2000-2002, 2015-2019)
- High volatility regimes (frequent dips)
- Overvalued markets (low forward return expectations)

**Verdict:** Elegant but not effective in trending bull markets. Stick to immediate pro-rata reinvestment for simplicity and performance.

### 7.4 The Tax Problem (Critical Limitation)

This entire analysis **did NOT model taxes**, which is a massive limitation for taxable accounts.

**Impact of taxes on trimming:**

Phase 3 best trimming strategy:
- Pre-tax: 21.36% CAGR → $670,744 final value
- Post-tax (estimated): ~18.0% CAGR → ~$550,000 final value
- Each trim: 15-20% LTCG tax on gains
- 14 trims over 10 years = substantial tax drag

Buy-and-hold:
- Pre-tax: 21.69% CAGR → $688,711 final value
- Post-tax: ~19.5% CAGR → ~$630,000 final value
- Single tax event at end (15-20% on total gain)
- Defers taxes for full decade

**Post-tax gap widens from -2.6% to ~-12.7%.**

**Implication:** In taxable accounts, trimming is even less attractive. Only makes sense in tax-deferred retirement accounts.

### 7.5 Limitations of This Analysis

**What was NOT modeled:**
1. **Taxes:** Each trim creates taxable event (15-20% LTCG significantly impacts results)
2. **Transaction costs:** Assumed $0 commissions (realistic today but not historically)
3. **Dividends:** Not included (would slightly favor buy-and-hold)
4. **Slippage:** Assumed perfect execution at close prices
5. **Single time period:** Only tested 2015-2024 (bull market with brief corrections)
6. **Survivorship bias:** All tickers still exist in 2024 (didn't test failing companies)

**Assumptions to question:**
1. Is 60/40 truly "typical"? (Many investors hold 80%+ index)
2. Why 20% trim amount? (10% or 30% might be better)
3. Cost basis reset at 1.05x - arbitrary choice
4. Pro-rata reinvestment assumes fractional shares
5. No modeling of portfolio additions (monthly contributions)

**Critical caveat:** These results are specific to 2015-2024, a decade-long bull market. Testing on 2000-2010 (bear markets, sideways action) would likely show different results favoring trimming.

### 7.6 Future Research Directions

To strengthen these findings, future analysis could:

**1. Test different time periods**
- 2000-2010: Dot-com crash + financial crisis
- 1990-2000: Dot-com bubble
- 1970-1980: Stagflation era
- Compare bull vs bear vs sideways markets

**2. Model tax drag explicitly**
- Calculate after-tax returns in taxable accounts
- Compare to tax-deferred (IRA/401k) results
- Test tax-loss harvesting strategies

**3. Test different trim percentages**
- 10%, 30%, 50% position sales
- Variable trim based on gain magnitude
- Trim more aggressively at higher valuations

**4. Dynamic/adaptive strategies**
- Adjust thresholds based on market P/E ratios
- Use volatility to inform trim timing
- Momentum-based trim signals

**5. Other asset classes**
- Bonds, commodities, REITs, crypto
- Multi-asset portfolios
- International equity exposure

**6. Machine learning approaches**
- Train models to predict optimal trim timing
- Adaptive strategies based on market regime
- Reinforcement learning for portfolio management

## 8. Final Thoughts

This investigation began with a simple question: **Does portfolio trimming beat buy-and-hold?**

After three phases of backtesting, analyzing 13 strategies across two portfolio compositions, and examining nearly 10 years of real market data, the answer is definitively: **It depends on what you own.**

### Three Lessons Learned

**1. Outliers dominate everything in investing**

One stock - NVIDIA with its 28,057% gain - completely invalidated the trimming thesis in Phase 1. The portfolio went from equal-weight allocation to 82% NVDA by final value. In concentrated portfolios, rare events matter infinitely more than average outcomes.

This aligns with what we know about stock returns: most stocks underperform, a tiny minority drive all market gains, and missing the best performers devastates returns. Trimming strategies, by definition, reduce exposure to winners. When one of those winners becomes a 100x+ outlier, that reduction becomes catastrophically expensive.

**The takeaway:** If you have high conviction in a potential outlier, let it run. The cost of being wrong (holding a loser too long) is far less than the cost of missing a 100x winner.

**2. Assumptions matter more than analysis**

The most consequential decision in this entire investigation wasn't choosing a trim threshold or reinvestment mode - it was **choosing the Phase 3 portfolio allocation.** 

By questioning the assumption that I'd owned NVDA at $0.48, I discovered that the entire conclusion flipped. Phase 1 showed -20.6% underperformance for trimming. Phase 3 showed only -2.6% underperformance with better risk metrics.

Same strategies. Same time period. Same backtest code. **Different portfolio composition = completely different conclusions.**

**The takeaway:** Garbage in, garbage out. The most important investment decisions happen before you open a brokerage account: asset allocation, portfolio construction, and what you choose to own. Strategy optimization is secondary.

**3. There's no universal best strategy**

Anyone claiming a one-size-fits-all investing approach is either ignorant or selling something.

The answer to "Should I trim?" isn't yes or no - it's:
- What do you own?
- What's your time horizon?
- What's your risk tolerance?
- Are you in a taxable or tax-deferred account?
- What market regime are you in?
- What are you optimizing for (max returns or smooth ride)?

For a 30-year-old with a concentrated tech portfolio in a Roth IRA optimizing for maximum wealth at 65: **Don't trim. Let winners run.**

For a 55-year-old with a 60/40 index portfolio in a taxable account optimizing for smooth retirement glide path: **Trimming is viable. The risk reduction might be worth it.**

**The takeaway:** Context matters. Your strategy should match your goals, constraints, and portfolio composition.

### Personal Reflection

For me, the Phase 3 results are most relevant. I don't own concentrated positions with 100x potential. I hold a diversified portfolio weighted toward index funds with selective stock picks. 

In that context, a systematic trimming strategy at +100% thresholds with pro-rata reinvestment offers:
- Nearly identical returns (21.4% vs 21.7% CAGR)
- Materially better Sharpe ratio (0.94 vs 0.90)
- Significantly lower maximum drawdown (-40.8% vs -46.3%)

That's a trade worth making. Giving up 0.3% annual return in exchange for 5.5% less maximum pain and 4.4% better risk-adjusted returns aligns with my goals and personality.

Your mileage may vary.

### The Bigger Picture

This investigation taught me that investing is full of trade-offs with no obvious answers:

- **Concentration vs diversification:** Own 10 stocks and maybe get a 100x winner, or own 50 and definitely avoid blowing up?
- **Maximum returns vs smooth ride:** Optimize CAGR or optimize sleep quality?
- **Systematic rules vs discretion:** Follow a mechanical strategy or make judgment calls?
- **Taxes vs tactics:** Implement optimal strategy or minimize tax drag?

There are no universal right answers. Only choices that align with your situation.

The best I can do is provide data, analysis, and transparency about assumptions and limitations. The rest is up to you.

### Closing Thought

If you're reading this and wondering "Should I trim my portfolio?", here's my advice:

**Ask yourself three questions:**

1. **Do I own potential 10x-100x outliers?**
   - Yes → Don't trim. Let them run.
   - No/Unsure → Proceed to question 2.

2. **Am I optimizing for maximum returns or smooth ride?**
   - Maximum returns → Don't trim. Accept volatility.
   - Smooth ride → Proceed to question 3.

3. **Is this a tax-deferred account (IRA/401k)?**
   - Yes → Trimming is viable with minimal cost.
   - No (taxable) → Tax drag makes trimming expensive. Reconsider.

If you answered "No, Smooth ride, Yes" - then yes, consider systematic trimming at +100% thresholds with pro-rata reinvestment.

Otherwise, just buy and hold.

---

**The simplest strategy is often the best strategy.**

---

## Appendix: Complete Results Tables

### Phase 1 (NVDA-Dominated Portfolio) - All Strategies

In [ ]:
# Complete Phase 1 results
print("=" * 120)
print("PHASE 1 COMPLETE RESULTS")
print("Portfolio: Equal-weight AAPL, MSFT, NVDA, TSLA, SPY, QQQ (16.7% each)")
print("Time Period: Jan 2015 - Nov 2024 (9.85 years)")
print("=" * 120)
display(results_phase1.sort_values('final_value', ascending=False).style.format({
    'total_return': '{:.2f}x',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'sortino_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'volatility': '{:.2%}',
    'final_value': '${:,.0f}',
    'num_trades': '{:.0f}',
    'cash_held': '${:,.0f}'
}).background_gradient(subset=['final_value'], cmap='RdYlGn'))

### Phase 3 (Realistic 60/40 Portfolio) - All Strategies

In [ ]:
# Complete Phase 3 results
print("=" * 120)
print("PHASE 3 COMPLETE RESULTS")
print("Portfolio: 60% Index (SPY 30%, QQQ 20%, VOO 10%) + 40% Stocks (AAPL 15%, MSFT 15%, TSLA 10%)")
print("Time Period: Jan 2015 - Nov 2024 (9.85 years)")
print("=" * 120)
display(results_phase3.sort_values('final_value', ascending=False).style.format({
    'total_return': '{:.2f}x',
    'cagr': '{:.2%}',
    'sharpe_ratio': '{:.3f}',
    'sortino_ratio': '{:.3f}',
    'max_drawdown': '{:.2%}',
    'volatility': '{:.2%}',
    'final_value': '${:,.0f}',
    'num_trades': '{:.0f}',
    'cash_held': '${:,.0f}'
}).background_gradient(subset=['final_value'], cmap='RdYlGn'))

### Metric Definitions

**Total Return:** $(\text{Final Value} - \text{Initial Value}) / \text{Initial Value}$

**CAGR (Compound Annual Growth Rate):** $(\text{Final Value} / \text{Initial Value})^{1/\text{Years}} - 1$

**Sharpe Ratio:** $\text{Return} / \text{Volatility}$ (higher is better, >1.0 is excellent)

**Sortino Ratio:** $\text{Return} / \text{Downside Deviation}$ (similar to Sharpe but only penalizes downside)

**Max Drawdown:** Largest peak-to-trough decline (measures worst-case pain)

**Volatility:** Annualized standard deviation of daily returns

**Num Trades:** Total number of trim events over the period

**Cash Held:** Cash balance at end of period (only relevant for "cash" reinvestment mode)

---

## Reproducibility

All code, data, and visualizations are available at:

**Project Directory:** `/Users/austinwallace/sandbox/stock_strategies/trim_strat_test/`

**Key Files:**
- `run_backtest_manual_data.py` - Phase 1 backtest engine (NVDA-dominated)
- `run_backtest_index_focus.py` - Phase 3 backtest engine (realistic 60/40)
- `generate_visualizations.py` - Chart generation script
- `manual_data/*.csv` - Historical price data from Yahoo Finance
- `results_real_data/real_data_results.csv` - Phase 1 results
- `results_index_focus/index_focus_results.csv` - Phase 3 results
- `visualizations/*.png` - All charts (300 DPI)

**To Reproduce Results:**
```bash
# From project directory
python run_backtest_manual_data.py       # Phase 1
python run_backtest_index_focus.py       # Phase 3
python generate_visualizations.py        # Charts
```

**Python Environment:**
```bash
pip install pandas numpy matplotlib seaborn yfinance
```

**Data Source:** Yahoo Finance historical prices (2015-2024)

**License:** Analysis and code for personal/educational use

---

*Report Generated: November 2024*  
*Author: Austin Wallace*  
*Research Period: January 2015 - November 2024 (9.85 years)*